# 动漫风格图像退化 (2× 轻量版) — HR → LR

在原有 4× 退化基础上做**更轻的退化**，更贴近 Diffusion 生成的壁纸粗图：
- **2× 下采样**（保留更多细节，避免 4× 丢太多信息）
- **更弱的中值滤波**（kernel 3×3，减少对细小细节的抹除）
- **更弱的高斯模糊与 USM 锐化**（小 kernel、小 sigma、小 amount）

输出保存到 **`dataset/lowres_2x/original`**，文件名与 `degradation_pipeline.ipynb` 一致（同名、png 转 jpg 保存）。

In [4]:
import os
import glob
import cv2
import numpy as np
import torch
import torch.nn.functional as F
import math

hr_dir = 'dataset/highres/original'
lr_dir = 'dataset/lowres_2x/original'

os.makedirs(lr_dir, exist_ok=True)

hr_image_paths = sorted(glob.glob(os.path.join(hr_dir, '*.[jp][pn]*g')))
print(f"总共找到 {len(hr_image_paths)} 张 HR 图片。")

总共找到 434 张 HR 图片。


In [5]:
def get_gaussian_kernel(kernel_size=3, sigma=0.8, channels=3):
    x_coord = torch.arange(kernel_size)
    x_grid = x_coord.repeat(kernel_size).view(kernel_size, kernel_size)
    y_grid = x_grid.t()
    xy_grid = torch.stack([x_grid, y_grid], dim=-1).float()
    mean = (kernel_size - 1) / 2.
    variance = sigma ** 2.
    gaussian_kernel = (1. / (2. * math.pi * variance)) * torch.exp(
        -torch.sum((xy_grid - mean) ** 2., dim=-1) / (2 * variance)
    )
    gaussian_kernel = gaussian_kernel / torch.sum(gaussian_kernel)
    gaussian_kernel = gaussian_kernel.view(1, 1, kernel_size, kernel_size)
    gaussian_kernel = gaussian_kernel.repeat(channels, 1, 1, 1)
    return gaussian_kernel

def gaussian_blur_pt(img_tensor, kernel_size=3, sigma=0.8):
    channels = img_tensor.shape[1]
    kernel = get_gaussian_kernel(kernel_size, sigma, channels).to(img_tensor.device)
    padding = kernel_size // 2
    return F.conv2d(img_tensor, kernel, padding=padding, groups=channels)

def median_blur_pt(img_tensor, kernel_size=3):
    B, C, H, W = img_tensor.shape
    padding = kernel_size // 2
    unfolded = F.unfold(img_tensor, kernel_size=kernel_size, padding=padding)
    unfolded = unfolded.view(B, C, kernel_size * kernel_size, H, W)
    median_tensor, _ = torch.median(unfolded, dim=2)
    return median_tensor

def degrade_image_2x_light(img_np, scale=2, device='cuda'):
    """
    2× 轻量退化：弱中值、弱模糊、弱 USM，尽量模仿 Diffusion 壁纸粗图。
    img_np: (H,W,C) BGR, 0-255
    """
    # 1. 轻度中值滤波 (HR) — 3×3，保留更多细节
    img_median_hr = cv2.medianBlur(img_np, 3)
    
    # 2. 2× 下采样
    h, w = img_median_hr.shape[:2]
    lr_h, lr_w = h // scale, w // scale
    img_lr_cpu = cv2.resize(img_median_hr, (lr_w, lr_h), interpolation=cv2.INTER_CUBIC)
    
    # 3. 转 Tensor 上 GPU
    img_rgb = cv2.cvtColor(img_lr_cpu, cv2.COLOR_BGR2RGB)
    img_tensor = torch.from_numpy(img_rgb).float().permute(2, 0, 1).unsqueeze(0) / 255.0
    img_tensor = img_tensor.to(device)
    
    # 4. 轻度二次中值 (LR) — 3×3
    img_lr_gpu = median_blur_pt(img_tensor, kernel_size=3)
    
    # 5. 轻度高斯模糊 — 3×3, sigma=0.8
    img_smooth = gaussian_blur_pt(img_lr_gpu, kernel_size=3, sigma=0.8)
    
    # 6. 轻度 USM 锐化 — 小 amount
    img_blur_for_usm = gaussian_blur_pt(img_smooth, kernel_size=3, sigma=1.0)
    amount = 0.8
    img_sharp = img_smooth + (img_smooth - img_blur_for_usm) * amount
    img_sharp = torch.clamp(img_sharp, 0.0, 1.0)
    
    out_tensor = img_sharp.squeeze(0).permute(1, 2, 0).cpu().numpy()
    out_bgr = cv2.cvtColor((out_tensor * 255.0).astype(np.uint8), cv2.COLOR_RGB2BGR)
    return out_bgr

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [6]:
lr_image_paths = []
for hr_path in hr_image_paths:
    filename = os.path.basename(hr_path)
    lr_path = os.path.join(lr_dir, filename)
    
    img_hr = cv2.imread(hr_path)
    if img_hr is None:
        print(f"无法读取: {hr_path}")
        continue
    
    with torch.no_grad():
        img_lr = degrade_image_2x_light(img_hr, scale=2, device=device)
    
    if filename.lower().endswith('.png'):
        filename = filename[:-4] + '.jpg'
        lr_path = os.path.join(lr_dir, filename)
    
    cv2.imwrite(lr_path, img_lr, [int(cv2.IMWRITE_JPEG_QUALITY), 85])
    lr_image_paths.append(lr_path)
    
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

if lr_image_paths:
    print(f"成功处理并保存了 {len(lr_image_paths)} 张 LR 图片到 {lr_dir}。")

成功处理并保存了 434 张 LR 图片到 dataset/lowres_2x/original。
